In [3]:
from IPython.display import display, HTML, Image
display(HTML("<style>.container { width:85% !important; }</style>"))
%config IPCompleter.use_jedi=False

In [15]:
import os
from datasets import load_dataset
from evaluate import load

In [12]:
code_eval = load("code_eval")

In [13]:
os.environ["HF_ALLOW_CODE_EVAL"] = "1"

In [2]:
dset = load_dataset('cakiki/humaneval-codeparrot-small-eval', split='train')

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Using custom data configuration cakiki--humaneval-codeparrot-small-eval-9755818be8af2d7a


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /tf/data/cache/HF/datasets/cakiki___parquet/cakiki--humaneval-codeparrot-small-eval-9755818be8af2d7a/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8. Subsequent calls will reuse this data.


In [7]:
df = dset.to_pandas()

In [17]:
for i in df.itertuples():
    a = i    
    break

In [ ]:
references.append("\n" + test_func + "\n" + entry_point)

In [64]:
c = (df.itertuples())

In [70]:
a = next(c)

In [28]:
a.cleaned_generation.tolist()

['\n\ndef truncate_number(number: float) -> float:\n    """ Given a positive floating point number, it can be decomposed into\n    and integer part (largest integer smaller than given number) and decimals\n    (leftover part always smaller than 1).\n\n    Return the decimal part of the number.\n    >>> truncate_number(3.5)\n    0.5\n    """\n"""\n    pyasn1 - Copyright (C) 2010  Aron Krund\n    pedan - pauldakn\n    License: BSD\n\nThis is part of Psyco \nhttp://github.com/petramjenny/Psyco \n\n This is part of PyOMA project\n\n   pyOMA is free software: you can redistribute it and/or modify\n   it under the terms of the GNU General Public License as published by\n   the Free Software Foundation, either version 3 of the License, or\n   (at your option) any later version.\n\n   pyOMA is distributed',
 '\n\ndef truncate_number(number: float) -> float:\n    """ Given a positive floating point number, it can be decomposed into\n    and integer part (largest integer smaller than given numbe

In [36]:
a.cleaned_generation.tolist()[0]

'\n\ndef truncate_number(number: float) -> float:\n    """ Given a positive floating point number, it can be decomposed into\n    and integer part (largest integer smaller than given number) and decimals\n    (leftover part always smaller than 1).\n\n    Return the decimal part of the number.\n    >>> truncate_number(3.5)\n    0.5\n    """\n"""\n    pyasn1 - Copyright (C) 2010  Aron Krund\n    pedan - pauldakn\n    License: BSD\n\nThis is part of Psyco \nhttp://github.com/petramjenny/Psyco \n\n This is part of PyOMA project\n\n   pyOMA is free software: you can redistribute it and/or modify\n   it under the terms of the GNU General Public License as published by\n   the Free Software Foundation, either version 3 of the License, or\n   (at your option) any later version.\n\n   pyOMA is distributed'

In [76]:
%%time
pass_at_k, meta = code_eval.compute(
            references=["\n" + a.test + "\n" + a.entry_point], predictions=[a.cleaned_generation.tolist()], num_workers=1, k=[1,10,50,100,150,200,500]
        )

CPU times: user 793 ms, sys: 11.7 s, total: 12.4 s
Wall time: 29.7 s


In [47]:
meta

defaultdict(list,
            {0: [(0,
               {'task_id': 0,
                'passed': False,
                'result': 'failed: EOF while scanning triple-quoted string literal (<string>, line 42)',
                'completion_id': 0}),
              (1,
               {'task_id': 0,
                'passed': False,
                'result': 'failed: EOF while scanning triple-quoted string literal (<string>, line 46)',
                'completion_id': 1}),
              (2,
               {'task_id': 0,
                'passed': True,
                'result': 'passed',
                'completion_id': 2}),
              (3,
               {'task_id': 0,
                'passed': False,
                'result': 'failed: EOF while scanning triple-quoted string literal (<string>, line 44)',
                'completion_id': 3}),
              (4,
               {'task_id': 0,
                'passed': False,
                'result': "failed: No module named 'homeassistant'",
   

In [75]:
pass_at_k

{'pass@1': 0.6599999999999998,
 'pass@10': 0.9999870494709188,
 'pass@50': 1.0,
 'pass@100': 1.0,
 'pass@150': 1.0,
 'pass@200': 1.0}

In [50]:
7000*15/3600

29.166666666666668